# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [65]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [66]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,15.29,52,0,3.05,AU,2023-05-26
1,1,hithadhoo,-0.6000,73.0833,24.42,91,100,3.15,MV,2023-05-26
2,2,tiksi,71.6872,128.8694,0.80,91,100,4.47,RU,2023-05-26
3,3,thompson,55.7435,-97.8558,9.09,93,100,8.75,CA,2023-05-26
4,4,hamilton,39.1834,-84.5333,11.66,64,0,4.12,US,2023-05-26


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [68]:
# Configure the map plot
Map = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles = "EsriImagery",frame_width = 800,
                                 frame_height = 600, color = "City", 
                                 size = (city_data_df["Humidity"]), alpha = 0.75)


# Display the map
Map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [69]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) & 
                                 (city_data_df["Wind Speed"]<4.5)
                                & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df.info()
city_data_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 16 to 437
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     16 non-null     int64  
 1   City        16 non-null     object 
 2   Lat         16 non-null     float64
 3   Lng         16 non-null     float64
 4   Max Temp    16 non-null     float64
 5   Humidity    16 non-null     int64  
 6   Cloudiness  16 non-null     int64  
 7   Wind Speed  16 non-null     float64
 8   Country     16 non-null     object 
 9   Date        16 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 1.4+ KB


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,sao vicente,-23.9631,-46.3919,23.10,26,0,1.68,BR,2023-05-26
42,42,holualoa,19.6228,-155.9522,21.91,95,0,3.60,US,2023-05-26
88,88,menongue,-14.6585,17.6910,21.13,42,0,0.66,AO,2023-05-26
115,115,kailua-kona,19.6406,-155.9956,23.98,92,0,3.60,US,2023-05-26
124,124,tamanrasset,22.7850,5.5228,22.95,13,0,1.54,DZ,2023-05-26


### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Lat","Lng","Humidity","Country"]].copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "NaN"
hotel_df=hotel_df.reset_index(drop = True) 
# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
0,bareilly,28.3500,79.4167,56,IN,NaN
1,mbamba bay,-11.2833,34.7667,61,TZ,NaN
2,cooper's town,26.8667,-77.5167,73,BS,NaN
3,shasta lake,40.6804,-122.3708,49,US,NaN
4,sidon,33.5631,35.3689,73,LB,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.iloc[index,1]
    longitude = hotel_df.iloc[index,2]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    categories = "accommodation.hotel"
    filters = f'circle:{longitude},{latitude},{radius}'
    bias = f'proximity:{longitude},{latitude}'
    apiKey = geoapify_key
    
    params = {"categories":categories,
              "filter":filters,
            "bias":bias,
    "apiKey": apiKey}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bareilly - nearest hotel: No hotel found
mbamba bay - nearest hotel: Bio Camp lodge
cooper's town - nearest hotel: No hotel found
shasta lake - nearest hotel: Fairfield Inn & Suites Redding
sidon - nearest hotel: Couvent Terre Sainte
ipatinga - nearest hotel: Independência
hata - nearest hotel: No hotel found
dowlatabad - nearest hotel: هتل آفتاب اصفهان
serhetabat - nearest hotel: No hotel found
olney - nearest hotel: No hotel found
pirenopolis - nearest hotel: Pousada das Cavalhadas


,City,Lat,Lng,Humidity,Country,Hotel Name
0,bareilly,28.3500,79.4167,56,IN,No hotel found
1,mbamba bay,-11.2833,34.7667,61,TZ,Bio Camp lodge
2,cooper's town,26.8667,-77.5167,73,BS,No hotel found
3,shasta lake,40.6804,-122.3708,49,US,Fairfield Inn & Suites Redding
4,sidon,33.5631,35.3689,73,LB,Couvent Terre Sainte
5,ipatinga,-19.4683,-42.5367,39,BR,Independência
6,hata,26.7461,83.7403,64,IN,No hotel found
7,dowlatabad,32.7998,51.6955,22,IR,هتل آفتاب اصفهان
8,serhetabat,35.2799,62.3438,17,TM,No hotel found
9,olney,39.1532,-77.0669,37,US,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
Map = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles = "EsriImagery",frame_width = 800,
                                 frame_height = 600, size = (hotel_df["Humidity"]*2), color = "City", alpha = 0.75,hover_cols=['Hotel Name','Country'])


# Display the map
Map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Hotel Name,Country)